In [1]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, CuDNNLSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation,Lambda,Softmax,Reshape

from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import tensorflow as tf
import numpy as np


%matplotlib inline

class tensors_tookits(object):
    def __init__(self):
        None
        
    def argmax_tensor(self,x, axis):
        return K.argmax(x, axis)

    def argmin_tensor(self,x, axis):
        return K.argmin(x, axis)


    def one_hot_tensor(self,x, num_classes):
        #by : https://fdalvi.github.io/blog/2018-04-07-keras-sequential-onehot/
        return K.one_hot(K.cast(x, 'uint8'), num_classes)

    def all_top_k(self,input,k):

        flatten=K.flatten(input)
        global_top_k=tf.nn.top_k(flatten, k)
#         print('global topk values:',K.eval(global_top_k.values))
#         print('glaobal topk indices:',K.eval(global_top_k.indices))
        indices=global_top_k.indices

        indices_row= K.cast(tf.floor(indices/input.shape[-1]),dtype='int32') 
    #     K.eval(indices_row)


        indices_col=indices%input.shape[-1] # dtype='int32'
        # indices_col=tf.mod(indices,a.shape[-1]) #  tensorflow 的数学运算 https://blog.csdn.net/zywvvd/article/details/78593618

    #     K.eval(indices_col)

        indices=K.concatenate( [K.reshape(indices_row,(1,indices_row.shape[0])) , K.reshape(indices_col,(1,indices_col.shape[0]))] , axis=0)
        indices=K.transpose(indices)

        return indices


class numpy_tookits(object):
    def __init__(self):
        None
        
    ##--part1--: 使用 numpy 复现 tf.nn.topk  ## 
    def topk_array(self,matrix, k, axis=1):
        """
        perform topK based on np.argsort
        :param matrix: to be sorted
        :param K: select and sort the top K items
        :param axis: dimension to be sorted.
        :return:
        """
        full_sort = np.argsort(matrix, axis=axis)
        return full_sort[ :,-k:]

    
    def partition_topk_array(self,matrix, K, axis=1):
        """
        perform topK based on np.argpartition
        :param matrix: to be sorted
        :param K: select and sort the top K items
        :param axis: 0 or 1. dimension to be sorted.
        :return: 
        """
        a_part = np.argpartition(matrix, -K, axis=axis)
        if axis == 0:
            row_index = np.arange(matrix.shape[1 - axis])
            a_sec_argsort_K = np.argsort(matrix[a_part[-K:, :], row_index], axis=axis)
            return a_part[-K:, :][a_sec_argsort_K, row_index]
        else:
            column_index = np.arange(matrix.shape[1 - axis])[:, None]
    #         print('column_index ',column_index)
    #         print('matrix[column_index, a_part[:, -K:]] ',matrix[column_index, a_part[:, -K:]]) #选取矩阵中的一组元素
            a_sec_argsort_K = np.argsort(matrix[column_index, a_part[:, -K:]], axis=axis)
    #         print('a_sec_argsort_K ',a_sec_argsort_K)
            return a_part[:, -K:][column_index, a_sec_argsort_K] # 乾坤大挪移，变换矩阵中的元素位置
    
    ##--part2--: arg_topK 输出 二维矩阵的 每一行 的topk，我们希望能输出整个矩阵的 topk
    def whole_topk_array(self,input,k):
        """
        输出 input 中所有元素中的 k 个最大的元素的下标，但是这k个元素并不会按照大小排序
        """

        flatten=input.flatten()
    #     print(flatten)

        global_top_k=np.argpartition(flatten, -k)[-k:]

        indices=global_top_k

        indices_row= np.floor(indices/input.shape[-1])


        indices_col=indices%input.shape[-1] # dtype='int32'


        indices=np.concatenate( [np.reshape(indices_row,(1,indices_row.shape[0])) , np.reshape(indices_col,(1,indices_col.shape[0]))] , axis=0)
        indices=np.transpose(indices)
        

        return indices.astype(np.int32) # numpy 数据类型转换 ；查看数据类型： arr.dtype
    
    
    ##--part3--: 使用 numpy 复现   tf.one_hot()
    def one_hot_array(self,targets, nb_classes):
        res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    #     print(res.shape)
        return res.reshape(list(targets.shape)+[nb_classes])


    
if __name__=='__main__':
    
    np_tool=numpy_tookits()
    tensors_tool=tensors_tookits()
    ##--part1--:  
    k=3
#     arr = np.array([[1, 98, 2, 99, 100]])

    a=np.array(
    [[1,2,3,4,5],
    [1,2,8,2,2],
    [9,3,3,3,6]]    
    )
    
    
#     np_tool.partition_topk_array(a, k, axis=1) 
#     np_tool.partition_topk_array(arr, k, axis=1)# 最大的 三个元素 (已排序)

    ## --ref: 
    # https://stackoverflow.com/questions/6910641/how-do-i-get-indices-of-n-maximum-values-in-a-numpy-array
    # https://stackoverflow.com/questions/41484104/how-numpy-partition-work
    # https://blog.csdn.net/SoftPoeter/article/details/86629329
    ##--part1-- end --##

    ##--part2--: arg_topK 输出 二维矩阵的 每一行 的topk，我们希望能输出整个矩阵的 topk


#     np_tool.whole_topk_array(a,k)
#     K.eval(tensors_tookits.all_top_k(a,k))

    ##--part2-- end --##

    ##--part3--: 使用 numpy 复现   tf.one_hot()

    # a.reshape(-1,3) # 固定3列 (-1)=多少行不知道，numpy自己算去吧
    # a.reshape(-1) #  flatten a 



#     a.shape
#     b=np_tool.one_hot_array(a,11)
#     b.shape

    ##--ref: 
    # https://stackoverflow.com/questions/38592324/one-hot-encoding-using-numpy
    ##--part3-- end --##


    
    
    
    

Using TensorFlow backend.
E:\anaconda\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
E:\anaconda\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
E:\anaconda\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
E:\anaconda\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
